In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


In [ ]:
# Sample synthetic dataset generation (for practice)
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'Temperature': np.random.normal(22, 5, n_samples),
    'Occupants': np.random.randint(1, 6, n_samples),
    'Appliance_Usage': np.random.normal(300, 100, n_samples),
    'Time_of_Day': np.random.choice(['Morning', 'Afternoon', 'Evening', 'Night'], n_samples),
})

# Simulate target variable with some logic
data['Electricity_Consumption'] = (
    0.5 * data['Temperature'] +
    20 * data['Occupants'] +
    0.8 * data['Appliance_Usage'] +
    np.random.normal(0, 30, n_samples)
)

data.head()


In [ ]:
data = pd.get_dummies(data, columns=['Time_of_Day'], drop_first=True)


In [ ]:
X = data.drop('Electricity_Consumption', axis=1)
y = data['Electricity_Consumption']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [ ]:
def evaluate_model(model, name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"🔍 {name} Results:")
    print("R² Score:", r2_score(y_test, y_pred))
    print("MSE:", mean_squared_error(y_test, y_pred))
    print("MAE:", mean_absolute_error(y_test, y_pred))
    print("-" * 40)
    
    return r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred), mean_absolute_error(y_test, y_pred)


In [ ]:
lr = LinearRegression()
evaluate_model(lr, "Linear Regression")


In [ ]:
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
y_pred_poly = poly_model.predict(X_test_poly)

print("🔍 Polynomial Regression")
print("R² Score:", r2_score(y_test, y_pred_poly))
print("MSE:", mean_squared_error(y_test, y_pred_poly))
print("MAE:", mean_absolute_error(y_test, y_pred_poly))


In [ ]:
ridge = Ridge(alpha=1.0)
evaluate_model(ridge, "Ridge Regression")


In [ ]:
lasso = Lasso(alpha=0.1)
evaluate_model(lasso, "Lasso Regression")


In [ ]:
tree = DecisionTreeRegressor(random_state=42)
evaluate_model(tree, "Decision Tree Regression")


In [ ]:
results = {
    "Model": ["Linear", "Polynomial", "Ridge", "Lasso", "Decision Tree", "Random Forest"],
    "R2 Score": [
        r2_score(y_test, lr.predict(X_test)),
        r2_score(y_test, y_pred_poly),
        r2_score(y_test, ridge.predict(X_test)),
        r2_score(y_test, lasso.predict(X_test)),
        r2_score(y_test, tree.predict(X_test)),
        r2_score(y_test, rf.predict(X_test))
    ]
}

results_df = pd.DataFrame(results)

plt.figure(figsize=(10,6))
sns.barplot(data=results_df, x='Model', y='R2 Score', palette='viridis')
plt.title("Model Comparison - R² Score")
plt.ylim(0, 1)
plt.show()


In [ ]:
params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20]
}

grid = GridSearchCV(RandomForestRegressor(random_state=42), params, cv=3, scoring='r2')
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)

best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)
print("Optimized Random Forest R²:", r2_score(y_test, y_pred_best))


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_best, alpha=0.6, color='teal')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("Actual Consumption")
plt.ylabel("Predicted Consumption")
plt.title("Actual vs Predicted Electricity Consumption")
plt.show()


In [ ]:
import joblib
joblib.dump(best_model, 'electricity_consumption_model.pkl')
